# 4bit & 8bit model inference

### NOTE: 4bit and 8bit quantization requires GPU

Load models in lower precision - 4bit or 8bit to reduce memory usage.

we use `bitsandbytes` library to load 4bit and 8bit models.

In [1]:
import torch
from transformers import AutoModelForCausalLM

In [2]:
MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.2'
# MODEL_NAME = 'google/gemma-2b-it'


### 4 bit model

In [3]:
model_4bit = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", load_in_4bit=True, torch_dtype="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [17]:
from transformers import BitsAndBytesConfig

use_4_bit_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
)

model_4bit = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", quantization_config=use_4_bit_config)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
print(model_4bit.get_memory_footprint())


4551360512


4 bit mistral 7B
* Model size: 4.5GB
* GPU mem used: 5.4 GB
* Loads on a T4 GPU

Use model

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
from transformers import pipeline

generator = pipeline(
    "text-generation", 
    model=model_4bit, 
    tokenizer=tokenizer,
    device_map="auto",
    # model_kwargs={"load_in_4bit": True}
)

In [15]:
print(generator(
    "Tell me a haiku about AI. ", 
    max_length=100,
    do_sample=True, 
    top_p=0.95
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Tell me a haiku about AI. 

Also, I've been thinking about a specific haiku related to AI that I've come up with but I've always worried if it is a real haiku since it has a sort of 5-7-5 pattern but it's really just a 2 syllable word (AI) in the last syllable. Is that a problem? Here is my Haiku:

Living in the


### Full precision model

In [6]:
!nvidia-smi

Mon Mar 25 21:04:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000001:00:00.0 Off |                  Off |
| N/A   32C    P8               9W /  70W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
full_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
print(full_model.get_memory_footprint())

30040678400


Full precision (16bit) mistral 7B
* Model size: 30 GB
* GPU mem used: xxx
* Loaded on CPU

In [7]:
full_model.device

device(type='cpu')